# Ekerå–Håstad's algorithm for factoring RSA integers
This notebook exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Ekerå–Håstads algorithm for factoring RSA integers [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20), with improvements from [[E20]](https://doi.org/10.1007/s10623-020-00783-2) and [[E23p]](https://doi.org/10.48550/arXiv.2309.01754), and the post-processing in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754).

To start off, let us pick two distinct prime factors $p$ and $q$ uniformly at random from the set of all $l$-bit prime factors, under the restriction that $N = pq$ is a $2l$-bit integer, so as to simulate how the modulus $N$ is typically selected in RSA. To this end, we use the [<code>sample_l_bit_prime(l)</code>](../docs/math/primes/sample_l_bit_prime.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [9]:
!pip3 install -q quaspy # Make sure that quaspy is installed.

In [12]:
from quaspy.math.primes import sample_l_bit_prime;

l = 1024;

while True:
  p = sample_l_bit_prime(l);

  while True:
    q = sample_l_bit_prime(l);
    if p != q:
      break;

  N = p * q;

  if (2 ** (2 * l - 1)) <= N < (2 ** (2 * l)):
    break;

print("Sampled p =", p);
print("Sampled q =", q);

print("\nComputed N =", N);

Sampled p = 177790946175996202448573953344264054514807356368958387963643292524671627693680903545445438389888848542540372201138683359521924826287651216980648892070261450531116119237056943005121911524311027834455467182229068316438076233167859847938683984456735896578836343253115690162766100459204868474546123438589151333919
Sampled q = 106975832459349338167515218133264508278122469545030318330801850161713234327992605292895424640957620940284657534222850480828563901923458215089523924987267776329454401152615383104586374333801588586753859138561719041426907883490240824403151655863393261534758674032596500811936573390712850126094903883179858274179

Computed N = 190193344709125656438968120885897474109367247318419465573096901644008608132896133661169061299650005815178677854377935135425020019731746458993742802680370342694961976614398433885639262979180640797218043688028300464301707850004392746547659195366476480427404204969732065431083367612139247265909020067502279570495169297384939547493836832654915985

## 1. Simulating order finding in $\mathbb Z_N^*$ exactly to sample $r$
The first step in the classical pre-processing in Ekerå–Håstad's factoring algorithm [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20) is to select $g$ uniformly at random from $\mathbb Z_N^*$.

[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) for exactly sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and returning its order without explicitly computing and returning $g$. This function requires that the factorization of $N$ is known, as is the case here given that we selected the factors of $N$.

Below, we use this function to simulate the initial classical pre-processing step of Ekerå–Håstad's algorithm. We then setup a simulated generator $g$ for a cyclic group of order $r$ by using the [SimulatedCyclicGroupElement](../docs/math/groups/SimulatedCyclicGroupElement.md) class provided by [Quaspy](https://github.com/ekera/quaspy).

In [13]:
from quaspy.factoring.sampling import sample_r_given_N;

from quaspy.math.groups import SimulatedCyclicGroupElement;

# Sample r.
r = sample_r_given_N(N, factors = [[p, 1], [q, 1]]);

print("Sampled r =", r);

# Setup a simulated cyclic group element of order r.
g = SimulatedCyclicGroupElement(r);

Sampled r = 1056629692828475869105378449366097078385373596213441475406093898022270045182756298117605896109166698973214876968766306307916777887398591438854126681557613014972010981191102410475773683217670226651211353822379447023898376944468848591931439974258202669041134472054067030172685375622995818143939000375012664280512897942214367178264866338556337218758530492328797023030208356390874033471584040885517852620999508504436991424092428102149207014666222329176687760119376524230731461134339382146789290177770638049369254624267224721513543252048106679556443398956851993692340665799247601641817097084626786692013751650342115276078


### 1.1. Simulating the quantum part of Ekerå–Håstad's algorithm
In Ekerå-Håstad's algorithm [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20), with improvements from [[E20]](https://doi.org/10.1007/s10623-020-00783-2) and [[E23p]](
https://doi.org/10.48550/arXiv.2309.01754), the idea is now to use $g$ to compute

$$x = g^{(N - 1) / 2 - 2^{l - 1}} = g^{((p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}}$$

classically by using that $N$ is known, and to then compute the discrete logarithm $\log_g x \in [0, r) \cap \mathbb Z$ quantumly.

Let $m = l - 1$ be an upper bound on the bit length of of the the logarithm, and let $\ell = m - \Delta$ for some positive constant $\Delta$.
Let $d = ((p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}$.
Provided that $r \ge 2^{m+\ell} + (2^\ell - 1)d$, we then have that $d = \log_g x$, in which case we can solve $N = pq$ and $d = ((p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}$ for $p$ and $q$ by solving a quadratic equation.

In the below example, $x$ is first computed given $N$. Furthermore, $d$ is computed given $p$ and $q$ since $d$ is needed by the simulator for the quantum part of Ekerå–Håstad's algorithm. [Quaspy](https://github.com/ekera/quaspy) provides convenience functions [<code>setup_x_given_g_N(g, N)</code>](../docs/factoring/rsa/setup_x_given_g_N.md) and [<code>setup_d_given_p_q(p, q)</code>](../docs/factoring/rsa/setup_d_given_p_q.md) for computing $x$ and $d$, respectively, but below we perform the computation explicitly to facilitate reader comprehension.

[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_k_given_d_r_tau(d, r, m, ell, tau)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r_tau.md) for simulating the quantum part of Ekerå–Håstad's factoring algorithm exactly (up to arbitrary precision) for a given logarithm $d$, order $r$, and parameters $m$, $\ell$ and $\tau$, where $\tau$ is related to the sampling as explained in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754). Below, we use the this function to simulate running the quantum part of Ekerå–Håstad's algorithm. The yields a frequency pair $(j, k)$ sampled from the probability distribution induced by the quantum algorithm.

Note that the [<code>sample_j_k_given_d_r_tau(d, r, m, ell, tau)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r_tau.md) function checks that $r \ge 2^{m+\ell} + (2^\ell - 1)d$. As explained in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754), this requirement is met with very large probability with $\Delta = 30$.

In [14]:
from quaspy.logarithmfinding.short.sampling import sample_j_k_given_d_r_tau;

# Compute r.
x = g ** ((N - 1) // 2 - 2 ** (l - 1));

# Compute d.
d = ((p - 1) // 2) + ((q - 1) // 2) - 2 ** (l - 1);

# Sanity check that x = g^d.
if x != g ** d:
  raise Exception("Error: Unexpected result.");

# Simulate running the quantum part of the algorithm for g and x.
m = l - 1;
Delta = 30;
ell = m - Delta;
tau = 20;

[j, k] = sample_j_k_given_d_r_tau(d, r, m, ell, tau);

print("Sampled j =", j);
print("Sampled k =", k);

Sampled j = 3974847159797048146688081283879881268934890262117519771160541746044361919385960817113197764019239054159939533289270817247736599460816723245930156464725239459057983657192766853988943044412326284290085490686289511315583412439332771463207878976275550258497945847728592949553083939472686301089834741228473638779695073809110359032043034076172640809228551370431510861599487144513651208696963996443267106662448214470711970696245000430496967743678302470647762508331480537228159023192391411142106893746807401172886170732957087174265132378814699827541538822409231250267106020718563243398402816577304167811724053138023
Sampled k = 15541199746928529165858827873857818888298093473090295741353877454738272949532946076260662241004369546107941617696444426293986261516796606898038029306776992839773423487891937675861045712959873300675195583996286873783117399823808991425142556030270338710568117538856854123577601112013607203295005877024


### 1.2. Solving the frequency pair $(j, k)$ for $p$ and $q$
We now proceed to solve the frequency pair $(j, k)$ for the factors $p$ and $q$ of $N$.

To this end, we first use the [<code>solve_j_k_for_d(j, m, l, g, N, ..)</code>](../docs/logarithmfinding/short/postprocessing/solve_j_k_for_d.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $(j, k)$ for $d$ using the lattice-based post-processing from [[E23p]](https://doi.org/10.48550/arXiv.2309.01754). We then solve $d$ for the prime factors $p$ and $q$ of $N$ using the post-processing in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754) that is also described in [[E20]](https://doi.org/10.1007/s10623-020-00783-2).

As stated above, to solve $d = ((p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}$ and $N = pq$ for $p$ and $q$, we only need to solve a quadratic equation.
[Quaspy](https://github.com/ekera/quaspy) provides a convenience function [<code>split_N_given_d(d, N)</code>](../docs/factoring/rsa/postprocessing/split_N_given_d.md) for this purpose, but below we solve explicitly using the quadratic formula to faciliate reader comprehension.

In [15]:
from quaspy.logarithmfinding.short.postprocessing import solve_j_k_for_d;

from gmpy2 import isqrt as sqrt;

# Recover d from the pair (j, k) output by the quantum algorithm.
for [tau, t] in [[4, 17], [7, 17], [14, 17], [17, 17], [20, 19]]:
  recovered_d = solve_j_k_for_d(j, k, m, ell, g, x, tau = tau, t = t);
  if recovered_d != None:
    break;

if None == recovered_d:
  print("[FAIL] Failed to recover d from (j, k).");
else:
  # Form d' = p + q given d.
  p_plus_q = 2 * recovered_d + 2 ** l + 2;

  # Solve d' = p + q and N = pq for p and q using the quadratic formula:
  candidate_p = (p_plus_q - sqrt((p_plus_q ** 2) - 4 * N)) // 2;
  candidate_q = (p_plus_q + sqrt((p_plus_q ** 2) - 4 * N)) // 2;

  if (1 < candidate_p < N) and \
    (1 < candidate_q < N) and \
    (candidate_p * candidate_q == N):
    print("p =", candidate_p);
    print("q =", candidate_q);

    print("\n[ OK ] Successfully recovered p and q.");
  else:
    print("[FAIL] Failed to recover p and q.");

p = 106975832459349338167515218133264508278122469545030318330801850161713234327992605292895424640957620940284657534222850480828563901923458215089523924987267776329454401152615383104586374333801588586753859138561719041426907883490240824403151655863393261534758674032596500811936573390712850126094903883179858274179
q = 177790946175996202448573953344264054514807356368958387963643292524671627693680903545445438389888848542540372201138683359521924826287651216980648892070261450531116119237056943005121911524311027834455467182229068316438076233167859847938683984456735896578836343253115690162766100459204868474546123438589151333919

[ OK ] Successfully recovered p and q.


## 2. Simulating order finding in $\mathbb Z_N^*$ heuristically to sample $g$ and $r$

In addition to the [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) function used above, [Quaspy](https://github.com/ekera/quaspy) also provides a function [<code>sample_g_r_given_N(N, N_factors, ..)</code>](../docs/factoring/sampling/sample_g_r_given_N.md) for sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and heuristically computing and returning its order $r$ alongside $g$.

This function requires that the factorization of $N = pq$ is known, as is the case here given that we selected $p$ and $q$. (If the function is also fed the factorization of $p - 1$ and $q - 1$ then the function can be made exact, as explained in the [documentation](../docs/factoring/sampling/sample_r_given_N.md).)

Below, we use this function to simulate the initial classical pre-processing step of Ekerå–Håstad's algorithm where an element $g$ is selected uniformly at random from $\mathbb Z_N^*$. This yields $g$ and $r$:

In [16]:
from quaspy.factoring.sampling import sample_g_r_given_N;
from quaspy.math.groups import IntegerModRingMulSubgroupElement;

[g, r] = sample_g_r_given_N(N, [[p, 1], [q, 1]]);

print("Sampled g =", g);
print("Sampled r =", r);

# Setup the group element g.
g = IntegerModRingMulSubgroupElement(g, N);

Sampled g = 16540857727947194892429953614140088993268814515150234657647856371268255653614233133103404487499219069162636428444273356485674123359175162115366423873023739603716283980812224083923576272268630448976975901240800642207570047655514997725477777169635236341474868579163588030473042383702484052958822385700215694684373755714127970335262731143211457122057636888528467023655899378415015196761427579152635612370703331743205612872445971160944355109070771247379627080656181950063689226775772255765570170939768915819146129812108085090555336821906370341883376016739533062524419106757178025087138711076014620383590808887641800623017
Sampled r = 3169889078485427607316135348098291235156120788640324426218281694066810135548268894352817688327500096919644630906298918923750333662195774316562380044672839044916032943573307231427321049653010679953634061467138341071695130833406545775794319922774608007123403416162201090518056126868987454431817001125037992841538693826643101534794599015669011656275591476986391

### 2.1. Simulating the quantum part of Ekerå–Håstad's algorithm
We may then proceed — in analogy with the above — to use the [<code>sample_j_k_given_d_r_tau(d, r, m, ell, tau)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r_tau.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to simulate the quantum part of Ekerå–Håstad's factoring algorithm exactly (up to arbitrary precision) for a given logarithm $d$, order $r$, and parameters $m$, $\ell$ and $\tau$, where $\tau$ is related to the sampling as explained in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754). This yields a frequency pair $(j, k)$ sampled from the probability distribution induced by the algorithm.

To first setup $x$ and $d$, we use the [<code>setup_x_given_g_N(g, N)</code>](../docs/factoring/rsa/setup_x_given_g_N.md) and [<code>setup_d_given_p_q(p, q)</code>](../docs/factoring/rsa/setup_d_given_p_q.md) convenience functions provided by [Quaspy](https://github.com/ekera/quaspy).

In [17]:
from quaspy.logarithmfinding.short.sampling import sample_j_k_given_d_r_tau;

from quaspy.factoring.rsa import setup_d_given_p_q;
from quaspy.factoring.rsa import setup_x_given_g_N;

# Compute x.
x = setup_x_given_g_N(g, N);

print("Computed x =", x);

# Compute d.
d = setup_d_given_p_q(p, q);

# Sanity check that x = g^d.
if x != g ** d:
  raise Exception("Error: Unexpected result.");

m = l - 1;
Delta = 30;
ell = m - Delta;
tau = 20;

[j, k] = sample_j_k_given_d_r_tau(d, r, m, l = ell, tau = tau);

print("\nSampled j =", j);
print("Sampled k =", k);

Computed x = 18896811055838750605947633620872743594718322989103319585061989969455677204934337111618487648661513532670231557805320130699475014666512871964394398681484663791719361525470431096479033562598534227882151292910189970853786985640175245284231972557499722202956710474549947591331821640601356392429142720580015527380122828582752152150976174576852254464845349840745650999618925721953317772276744734245139278981640444506728449487958617451333002711763792391594970978709865711055417589289406990385107114993251046986122949733332863044326677913889388865465155545535419775742242100985425423863346747362405519018486360561152357694415 (mod 1901933447091256564389681208858974741093672473184194655730969016440086081328961336611690612996500058151786778543779351354250200197317464589937428026803703426949619766143984338856392629791806407972180436880283004643017078500043927465476591953664764804274042049697320654310833676121392472659090200675022795704951692973849395474938368326549159850044647868783233512083

### 2.2. Solving the frequency pair $(j, k)$ for $p$ and $q$
Finally, we may proceed — again in analogy with the above — to solve the frequency pair $(j, k)$ for the factors $p$ and $q$ of $N$.

To this end, we first use the [<code>solve_j_k_for_d(j, m, l, g, N, ..)</code>](../docs/logarithmfinding/short/postprocessing/solve_j_k_for_d.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $(j, k)$ for $d$ using the lattice-based post-processing from [[E23p]](https://doi.org/10.48550/arXiv.2309.01754). We then solve $d$ and $N$ for the prime factors $p$ and $q$ of $N$ by using the [<code>split_N_given_d(d, N)</code>](../docs/factoring/rsa/postprocessing/split_N_given_d.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [18]:
from quaspy.logarithmfinding.short.postprocessing import solve_j_k_for_d;

from quaspy.factoring.rsa.postprocessing import split_N_given_d;

# Recover d.
for [tau, t] in [[4, 17], [7, 17], [14, 17], [17, 17], [20, 19]]:
  recovered_d = solve_j_k_for_d(j, k, m, ell, g, x, tau = tau, t = t);
  if recovered_d != None:
    break;

if None == recovered_d:
  print("[FAIL] Failed to recover d from (j, k).");
else:
  result = split_N_given_d(d, N);

  if None != result:
    [recovered_p, recovered_q] = result;

    print("p =", recovered_p);
    print("q =", recovered_q);

    print("\n[ OK ] Successfully recovered p and q.");
  else:
    print("[FAIL] Failed to recover p and q.");

p = 106975832459349338167515218133264508278122469545030318330801850161713234327992605292895424640957620940284657534222850480828563901923458215089523924987267776329454401152615383104586374333801588586753859138561719041426907883490240824403151655863393261534758674032596500811936573390712850126094903883179858274179
q = 177790946175996202448573953344264054514807356368958387963643292524671627693680903545445438389888848542540372201138683359521924826287651216980648892070261450531116119237056943005121911524311027834455467182229068316438076233167859847938683984456735896578836343253115690162766100459204868474546123438589151333919

[ OK ] Successfully recovered p and q.
